In [1]:
# Cell 1: Imports, Mixed Precision, GPU & DLL Setup, Configuration

import os, glob, ctypes
import h5py, numpy as np, pandas as pd, matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, Model, mixed_precision
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tqdm import tqdm
from sklearn.metrics import (
    confusion_matrix, precision_score, recall_score,
    f1_score, accuracy_score, mean_squared_error, mean_absolute_error
)

# Enable mixed precision for speed on GPU
mixed_precision.set_global_policy('mixed_float16')

# DLL fix for Windows HDF5/zlib
try:
    dll = os.path.join(os.environ['CONDA_PREFIX'], 'Library', 'bin', 'zlibwapi.dll')
    ctypes.CDLL(dll)
    print("Loaded zlibwapi.dll")
except Exception:
    print("Could not load zlibwapi.dll")

# GPU config
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus: tf.config.experimental.set_memory_growth(gpu, True)
    print(f"Using {len(gpus)} GPU(s)")
else:
    print("No GPU detected")

# Config — tuned for speed
DATA_DIR      = r"C:\college\CV\COSMOS\6C_full"
SEQ_LEN       = 4         # use 4 time steps instead of 6
PATCH_SIZE    = 32        # use 32×32 patches instead of 64×64
BATCH_SIZE    = 16        # larger batch if GPU memory allows
EPOCHS        = 10        # max epochs
THRESHOLD     = 265.0
CV_THRESHOLD  = 260.0
FOG_THRESHOLD = 270.0
MODEL_PATH    = r"C:\college\CV\COSMOS\multitask_nowcast_fast.h5"

# list files & build sliding windows
all_files = sorted(glob.glob(os.path.join(DATA_DIR, "*.h5")))
sequences = [all_files[i:i+SEQ_LEN+1] for i in range(len(all_files)-SEQ_LEN)]


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3060 Laptop GPU, compute capability 8.6
Loaded zlibwapi.dll
Using 1 GPU(s)


Data-Loading & Generator Functions

In [2]:
# Cell 2: Data Loader & Generator

def load_multi(fp_seq):
    frames = []
    for fp in fp_seq[:SEQ_LEN]:
        with h5py.File(fp,'r') as f:
            cnt1, cnt2   = f['IMG_TIR1'][0][...], f['IMG_TIR2'][0][...]
            cnt_wv, cnt_mir = f['IMG_WV'][0][...], f['IMG_MIR'][0][...]
            cnt_vis      = f['IMG_VIS'][0][...]
            lut1, lut2   = f['IMG_TIR1_TEMP'][:],  f['IMG_TIR2_TEMP'][:]
            lut_wv, lut_mir = f['IMG_WV_TEMP'][:], f['IMG_MIR_TEMP'][:]
            lut_vis      = f['IMG_VIS_ALBEDO'][:]
        bt1 = lut1[cnt1]; bt2 = lut2[cnt2]
        wv  = lut_wv[cnt_wv]; mir = lut_mir[cnt_mir]
        vis = lut_vis[cnt_vis]
        frames.append(np.stack([bt1,bt2,wv,mir,vis],axis=-1)/300.0)
    X = np.stack(frames,axis=0).astype(np.float32)

    with h5py.File(fp_seq[-1],'r') as f:
        cnt1, cnt2   = f['IMG_TIR1'][0][...], f['IMG_TIR2'][0][...]
        cnt_wv, cnt_mir = f['IMG_WV'][0][...], f['IMG_MIR'][0][...]
        lut1, lut2   = f['IMG_TIR1_TEMP'][:],  f['IMG_TIR2_TEMP'][:]
        lut_wv, lut_mir = f['IMG_WV_TEMP'][:], f['IMG_MIR_TEMP'][:]
    bt1_t = lut1[cnt1]; bt2_t = lut2[cnt2]
    wv_t  = lut_wv[cnt_wv]; mir_t = lut_mir[cnt_mir]

    # temperature trend normalized
    last_mean  = bt1_t.mean()/300.0
    first_mean = X[0,...,0].mean()
    temp_trend = np.array([last_mean - first_mean],dtype=np.float32)

    return X, {
        'cloud'          : (bt1_t<THRESHOLD).astype(np.float32)[...,None],
        'convective'     : (bt1_t<CV_THRESHOLD).astype(np.float32)[...,None],
        'fog'            : (mir_t<FOG_THRESHOLD).astype(np.float32)[...,None],
        'moisture'       : (wv_t/300.0).astype(np.float32)[...,None],
        'thermo_contrast': ((bt2_t-bt1_t)/100.0).astype(np.float32)[...,None],
        'temp_trend'     : temp_trend
    }

def random_crop(X,y):
    H,W = X.shape[1], X.shape[2]
    i,j = np.random.randint(0,H-PATCH_SIZE), np.random.randint(0,W-PATCH_SIZE)
    Xc = X[:,i:i+PATCH_SIZE,j:j+PATCH_SIZE,:]
    yc = {}
    for k,v in y.items():
        yc[k] = v[i:i+PATCH_SIZE,j:j+PATCH_SIZE] if v.ndim==3 else v
    return Xc,yc

def generator(seqs):
    while True:
        np.random.shuffle(seqs)
        for seq in seqs:
            X,y = load_multi(seq)
            yield random_crop(X,y)


Dataset Split & steps_per_epoch

In [3]:
# Cell 3: Dataset Split, Caching & Pipelines

# train/val split
split       = int(0.9*len(sequences))
train_seqs  = sequences[:split]
val_seqs    = sequences[split:]

# steps per epoch
train_steps = len(train_seqs)//BATCH_SIZE
val_steps   = len(val_seqs)//BATCH_SIZE

# common output signature
output_signature = (
    tf.TensorSpec((SEQ_LEN,PATCH_SIZE,PATCH_SIZE,5),tf.float32),
    {
      'cloud': tf.TensorSpec((PATCH_SIZE,PATCH_SIZE,1),tf.float32),
      'convective': tf.TensorSpec((PATCH_SIZE,PATCH_SIZE,1),tf.float32),
      'fog': tf.TensorSpec((PATCH_SIZE,PATCH_SIZE,1),tf.float32),
      'moisture': tf.TensorSpec((PATCH_SIZE,PATCH_SIZE,1),tf.float32),
      'thermo_contrast': tf.TensorSpec((PATCH_SIZE,PATCH_SIZE,1),tf.float32),
      'temp_trend': tf.TensorSpec((1,),tf.float32)
    }
)

train_ds = tf.data.Dataset.from_generator(
    lambda: generator(train_seqs), output_signature=output_signature
).cache().batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_generator(
    lambda: generator(val_seqs), output_signature=output_signature
).cache().batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


Cell 4: Model Definition & Compilation

In [4]:
# Cell 4: Model Definition & Compilation

inp = layers.Input((SEQ_LEN,PATCH_SIZE,PATCH_SIZE,5))
x = layers.ConvLSTM2D(32,(3,3),padding='same',return_sequences=True,activation='relu')(inp)
x = layers.BatchNormalization()(x)
x = layers.ConvLSTM2D(16,(3,3),padding='same',return_sequences=False,activation='relu')(x)
x = layers.BatchNormalization()(x)

heads = {
  'cloud'          : layers.Conv2D(1,(1,1),activation='sigmoid',   name='cloud')(x),
  'convective'     : layers.Conv2D(1,(1,1),activation='sigmoid',   name='convective')(x),
  'fog'            : layers.Conv2D(1,(1,1),activation='sigmoid',   name='fog')(x),
  'moisture'       : layers.Conv2D(1,(1,1),activation='linear',    name='moisture')(x),
  'thermo_contrast': layers.Conv2D(1,(1,1),activation='linear',    name='thermo_contrast')(x),
}
temp_avg = layers.GlobalAveragePooling2D()(x)
heads['temp_trend'] = layers.Dense(1,activation='linear',name='temp_trend')(temp_avg)

model = Model(inputs=inp, outputs=heads, name='fast_multitask_nowcast')
model.compile(
  optimizer='adam',
  loss={
    'cloud':'binary_crossentropy','convective':'binary_crossentropy','fog':'binary_crossentropy',
    'moisture':'mse','thermo_contrast':'mse','temp_trend':'mse'
  },
  loss_weights={'cloud':1,'convective':1,'fog':1,'moisture':0.5,'thermo_contrast':0.5,'temp_trend':0.1},
  metrics={'cloud':'accuracy','convective':'accuracy','fog':'accuracy'}
)
model.summary()


Model: "fast_multitask_nowcast"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 4, 32, 32,   0           []                               
                                5)]                                                               
                                                                                                  
 conv_lstm2d (ConvLSTM2D)       (None, 4, 32, 32, 3  42752       ['input_1[0][0]']                
                                2)                                                                
                                                                                                  
 batch_normalization (BatchNorm  (None, 4, 32, 32, 3  128        ['conv_lstm2d[0][0]']            
 alization)                     2)                                           

Training with Progress Bars

In [ ]:
# Cell 5: Training

checkpoint = ModelCheckpoint(
  MODEL_PATH, monitor='val_cloud_accuracy',
  save_best_only=False, verbose=1
)

history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=EPOCHS,
  steps_per_epoch=train_steps,
  validation_steps=val_steps,
  callbacks=[checkpoint],
  verbose=1
)


Epoch 1/10
50/50 [==============================] - ETA: 0s - loss: 2.1109 - cloud_loss: 0.5481 - convective_loss: 0.5631 - fog_loss: 0.6219 - moisture_loss: 0.6031 - temp_trend_loss: 0.0396 - thermo_contrast_loss: 0.1443 - cloud_accuracy: 0.7207 - convective_accuracy: 0.7386 - fog_accuracy: 0.6733 
Epoch 1: val_cloud_accuracy improved from -inf to 0.73488, saving model to C:\college\CV\COSMOS\multitask_nowcast_fast.h5
50/50 [==============================] - 3098s 62s/step - loss: 2.1109 - cloud_loss: 0.5481 - convective_loss: 0.5631 - fog_loss: 0.6219 - moisture_loss: 0.6031 - temp_trend_loss: 0.0396 - thermo_contrast_loss: 0.1443 - cloud_accuracy: 0.7207 - convective_accuracy: 0.7386 - fog_accuracy: 0.6733 - val_loss: 2.1309 - val_cloud_loss: 0.6645 - val_convective_loss: 0.6556 - val_fog_loss: 0.6675 - val_moisture_loss: 0.2864 - val_temp_trend_loss: 1.7972e-04 - val_thermo_contrast_loss: 0.0014 - val_cloud_accuracy: 0.7349 - val_convective_accuracy: 0.7774 - val_fog_accuracy: 0.79

Evaluation & Metrics Display

In [ ]:
# Cell 6 — Evaluation
seg_keys = ["cloud","convective","fog"]
reg_keys = ["moisture","thermo_contrast","temp_trend"]

y_true_seg, y_pred_seg = {k:[] for k in seg_keys}, {k:[] for k in seg_keys}
y_true_reg, y_pred_reg = {k:[] for k in reg_keys}, {k:[] for k in reg_keys}

for Xb, yb in tqdm(val_ds, desc="Evaluation"):
    preds = model(Xb, training=False)
    for k in seg_keys:
        y_true_seg[k].append(tf.reshape(yb[k], (-1,)).numpy())
        y_pred_seg[k].append((tf.reshape(preds[k], (-1,)) > 0.5).numpy())
    for k in reg_keys:
        y_true_reg[k].append(tf.reshape(yb[k], (-1,)).numpy())
        y_pred_reg[k].append(tf.reshape(preds[k], (-1,)).numpy())

for d in (y_true_seg, y_pred_seg, y_true_reg, y_pred_reg):
    for k in d: d[k] = np.concatenate(d[k])

# ── segmentation --------------------------------------------------------------
seg_rows=[]
for k in seg_keys:
    cm = confusion_matrix(y_true_seg[k], y_pred_seg[k])
    seg_rows.append({"Task":k,
                     "Acc":accuracy_score(y_true_seg[k], y_pred_seg[k]),
                     "Prec":precision_score(y_true_seg[k], y_pred_seg[k], zero_division=0),
                     "Rec":recall_score(y_true_seg[k], y_pred_seg[k], zero_division=0),
                     "F1":f1_score(y_true_seg[k], y_pred_seg[k], zero_division=0),
                     "TN":cm[0,0],"FP":cm[0,1],"FN":cm[1,0],"TP":cm[1,1]})
df_seg = pd.DataFrame(seg_rows).set_index("Task")
print("### Segmentation"); display(df_seg)

# ── regression ----------------------------------------------------------------
reg_rows=[]
for k in reg_keys:
    reg_rows.append({"Task":k,
                     "MSE":mean_squared_error(y_true_reg[k], y_pred_reg[k]),
                     "MAE":mean_absolute_error(y_true_reg[k], y_pred_reg[k])})
df_reg = pd.DataFrame(reg_rows).set_index("Task")
print("### Regression"); display(df_reg)

# ── confusion matrices --------------------------------------------------------
for k in seg_keys:
    cm = confusion_matrix(y_true_seg[k], y_pred_seg[k])
    plt.figure(figsize=(4,4))
    plt.title(f"{k.capitalize()} – Confusion matrix")
    plt.imshow(cm, cmap="Blues"); plt.xlabel("Pred"); plt.ylabel("True")
    for (i,j),v in np.ndenumerate(cm): plt.text(j,i,str(v), ha="center", va="center")
    plt.colorbar(); plt.show()
    

Eval Seg: 119it [1:43:57, 51.79s/it]